<a href="https://colab.research.google.com/github/PTC-Education/PTC-API-Playground/blob/main/Onshape_API_Snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. READ ME - Onshape API Snippets Overview
This snippets library is split into numbered sections to help you get started with interacting with Onshape's REST API using their [Python Client](https://github.com/onshape-public/onshape-clients/tree/master/python). 

***Start by using snippets 0.1 and either 0.2 or 0.3*** to import the client and configuring you connection with API Keys generated in [Onshape's Dev Portal](https://dev-portal.onshape.com/).
1. PartStudios
2. Parts
3. MetaData
4. FeatureStudios

# 0.1 Connect - Import Python Client
This snippet shows how to install the [python-client from Onshape](https://github.com/onshape-public/onshape-clients/tree/master/python)

In [ ]:
!pip install onshape-client
from onshape_client.client import Client
import json
base = 'https://cad.onshape.com' # change this if you're using a document in an enterprise (i.e. "https://ptc.onshape.com")

# 0.2 Connect - Configure with API Keys in snippet
Set the configuration with app keys generated from [the developer portal](https://dev-portal.onshape.com/). Replace the 'access-key-here' and 'secret-key-here' strings with your credentials.

In [ ]:
client = Client(configuration={"base_url": base,
                               "access_key": 'access-key-here',
                               "secret_key": 'secret-key-here'})

# 0.3 Connect - Configure with API Keys in separate document
This snippet shows how you can pull API keys from a document stored in your google drive. **Good for sharing a notebook but keeping your keys secret**. Run the first snippet to load files from drive, then upload a python file that includes your access and secret keys as strings to your drive (can upload from Colab with the folder icon on the left menu).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
def include(filename):
    if os.path.exists(filename): 
        execfile(filename)
include('colabkeys.py')
client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})

# 1.1 PartStudios - GET Mass Properties
For certain API endpoints, you must pass specific endpoints. For this example, you need to provide the element id of a part studio to get the mass properties for the parts in that part studio.

In [ ]:
fixed_url = '/api/partstudios/d/did/w/wid/e/eid/massproperties'

did = '4bda16c648566259ea1b4e4c'
wid = 'c299b9fc994574c2637e871d'
eid = 'f957091ae301cf546ac8dc4b'

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(json.loads(response.data))


# 1.1.1 PartStudios - Get Mass Properties and Calculate Density
Sample of using the mass properties endpoint to get information and do calculations on the data. Sample document here: https://cad.onshape.com/documents/e025dee9cd9e880498ecc623/w/67f35e2ecb147b5d6c4e4498/e/919a01d3d602070cc98c0802

In [ ]:
fixed_url = '/api/partstudios/d/did/w/wid/e/eid/massproperties'

# https://cad.onshape.com/documents/e025dee9cd9e880498ecc623/w/67f35e2ecb147b5d6c4e4498/e/919a01d3d602070cc98c0802
did = 'e025dee9cd9e880498ecc623'
wid = '67f35e2ecb147b5d6c4e4498'
eid = '919a01d3d602070cc98c0802'

method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(json.loads(response.data))
# parse json to get mass and volume for all parts in the document
volume = json.loads(response.data)['bodies']['-all-']['volume'][0] # three element array is volume, max volume, and minimum volume
mass = json.loads(response.data)['bodies']['-all-']['mass'][0] # three element array is mass, maximum mass, and minimum mass

# calculate density with mass and volume in kg/m^3
density = mass/volume
print(density)

# 1.1.2 PartStudios - Get Mass Properties from Configured Part

In [ ]:
fixed_url = '/api/partstudios/d/did/w/wid/e/eid/massproperties'

#https://cad.onshape.com/documents/bb1a0fa7536a74f282879c0a/w/107d615fca7419084dd2ddf3/e/1cb13d9abf5bab6bcb8e3668
did = 'bb1a0fa7536a74f282879c0a'
wid = '107d615fca7419084dd2ddf3'
eid = '1cb13d9abf5bab6bcb8e3668'

method = 'GET'

# add configureation to the params of the request
params = {'configuration': 'Length%3D0.2%2Bmeter'}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(json.loads(response.data))

# 1.1.3 PartStudios - Get Mass Prop for series of configurations and plot

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

fixed_url = '/api/partstudios/d/did/w/wid/e/eid/massproperties'

#https://cad.onshape.com/documents/bb1a0fa7536a74f282879c0a/w/107d615fca7419084dd2ddf3/e/1cb13d9abf5bab6bcb8e3668
did = 'bb1a0fa7536a74f282879c0a'
wid = '107d615fca7419084dd2ddf3'
eid = '1cb13d9abf5bab6bcb8e3668'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

method = 'GET'
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
print(json.loads(response.data))
x = []
y = []

for i in range(1,5):
  x.append(i)
  configdef = 'Length%3D'+str(i)+'%2Bmeter'
  params = {'configuration': configdef}

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
  massProp = json.loads(response.data)
  y.append(massProp['bodies']['-all-']['inertia'][0])

  # The command below prints the entire JSON response from Onshape
  print(massProp['bodies']['-all-']['inertia'][0])
plt.plot(x, y, 'o', color='black');


# 1.2 PartStudios - Make Part Studio
This is an example of a post request that creates a new part studio in an Onshape document.

In [ ]:
fixed_url = '/api/partstudios/d/did/w/wid'

did = '4bda16c648566259ea1b4e4c'
wid = 'c299b9fc994574c2637e871d'

method = 'POST'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(json.loads(response.data))

# 2.1 Parts - Get All Parts in Document
This snippet shows a sample of getting all of the parts in an Onshape document. 

In [ ]:
# The first part of the fixed URL specifies which API we're calling
fixed_url = '/api/parts/d/did/w/wid'
# The parameters below are found in the URL of your onshape document. i.e. "https://cad.onshape.com/documents/4bda16c648566259ea1b4e4c/w/c299b9fc994574c2637e871d/e/34431211dd542fb7cc570959"
did = '4bda16c648566259ea1b4e4c'
wid = 'c299b9fc994574c2637e871d'

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
# print(json.loads(response.data))

parts = json.loads(response.data)
for i in range(len(parts)):
  print(parts[i]["name"] +" has part ID: "+ parts[i]["partId"])

# 3.1 MetaData - Update Part Color
The function below gets the metadata for a part, then changes the part's color and opacity, and sends a post request to update it. The function need a part id (pid) which can be found with the snippet in 2.4 above.

In [ ]:
def updateColor(eid,pid,newOpacity,newColors):

  fixed_url = '/api/metadata/d/did/w/wid/e/eid/p/pid'
  did = '4bda16c648566259ea1b4e4c'
  wid = 'c299b9fc994574c2637e871d'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', did)
  fixed_url = fixed_url.replace('wid', wid)
  fixed_url = fixed_url.replace('eid', eid)
  fixed_url = fixed_url.replace('pid', pid)

  response = client.api_client.request('GET', url=base + fixed_url,
  query_params=params, headers=headers, body=payload)

  partMetaData = json.loads(response.data)
  partProperties = partMetaData['properties']

  # # Print current appearance values
  # for x in partProperties:
  #   if x['name'] == "Appearance":
  #     partAppearance = x
  #     break
  # print(partAppearance['value'])

  for i in range(len(partProperties)):
    if partProperties[i]['name'] == "Appearance":
      partProperties[i]['value']['opacity'] = newOpacity
      partProperties[i]['value']['color'] = newColors
      break

  partMetaData['properties'] = partProperties

  payload = partMetaData

  response = client.api_client.request('POST', url=base + fixed_url,
  query_params=params, headers=headers, body=payload)

In [ ]:
updateColor('7b5bc60e32e112ae487c32dc','JMD',150,{'red': 255, 'green': 1, 'blue': 1})

# 4.1 FeatureStudios - Create Feature Studio
First snippet creates a FeatureStudio

In [ ]:
fixed_url = '/api/featurestudios/d/did/w/wid'

did = 'e025dee9cd9e880498ecc623'
wid = '67f35e2ecb147b5d6c4e4498'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)

method = 'POST'

params = {"name":"ColabFeature"}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# The command below prints the entire JSON response from Onshape
print(json.loads(response.data))

# 4.2 FeatureStudios - Get FeatureSpecs
Element ID must be for a FeatureStudio

In [ ]:
fixed_url = '/api/featurestudios/d/did/w/wid/e/eid/featurespecs'

did = 'e025dee9cd9e880498ecc623'
wid = '67f35e2ecb147b5d6c4e4498'
eid = '7e843682a5c2ec5c8a0bb1af'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
print(response.data)

# 4.3 FeatureStudios - Get Feature Content

In [ ]:
fixed_url = '/api/featurestudios/d/did/w/wid/e/eid'

did = 'e025dee9cd9e880498ecc623'
wid = '67f35e2ecb147b5d6c4e4498'
eid = '7e843682a5c2ec5c8a0bb1af'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
print(response.data)

# 4.4 FeatureStudios - Update Feature Content


In [ ]:
fixed_url = '/api/featurestudios/d/did/w/wid/e/eid'

did = 'e025dee9cd9e880498ecc623'
wid = '67f35e2ecb147b5d6c4e4498'
eid = '7e843682a5c2ec5c8a0bb1af'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

newBody = json.loads(response.data)
print(newBody["contents"]) #print current contents of featurestudio

#define new feature here
feature = "FeatureScript 1494;\nimport(path : \"onshape/std/geometry.fs\", version : \"1494.0\");\n\nannotation { \"Feature Type Name\" : \"My Feature\" }\nexport const myFeature = defineFeature(function(context is Context, id is Id, definition is map)\n    precondition\n    {\n        // Define the parameters of the feature type\n    }\n    {\n        // Define the function's action\n        var sketch1 = newSketch(context, id + \"sketch1\", {\n                \"sketchPlane\" : qCreatedBy(makeId(\"Top\"), EntityType.FACE)\n        });\n        // Create sketch entities here\n        skSolve(sketch1);\n        \n    });\n"

newBody["contents"] = feature
payload = newBody

method = 'POST'
response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

# 5.1 Elements - Get Configurations

In [ ]:
fixed_url = '/api/elements/d/did/w/wid/e/eid/configuration'
#https://cad.onshape.com/documents/bb1a0fa7536a74f282879c0a/w/107d615fca7419084dd2ddf3/e/1cb13d9abf5bab6bcb8e3668
did = 'bb1a0fa7536a74f282879c0a'
wid = '107d615fca7419084dd2ddf3'
eid = '1cb13d9abf5bab6bcb8e3668'

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

# Method below specifies the request type - 'GET' or 'POST'
method = 'GET'

params = {}
payload = {}
headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)
ConfigInfo = json.loads(response.data)
ConfigParams = ConfigInfo["configurationParameters"]

for i in range(len(ConfigParams)):
  print(ConfigParams[i]["message"]["parameterName"])
  print(ConfigParams[i]["message"]["rangeAndDefault"]["message"])